In [14]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import matplotlib.pylab as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [15]:
def img_to_array(img_name):
    img = Image.open("eaSF/all_letters_image/all_letters_image/" + img_name).convert('RGB')
    img=img.crop((0,0,32,32))
    x = np.array(img)
    return np.expand_dims(x, axis=0)
def data_to_tensor(img_names):
    list_of_tensors = [img_to_array(img_name) for img_name in img_names]
    return np.vstack(list_of_tensors)

In [16]:
data = pd.read_csv("all_letters_info.csv")
image_names = data['file']
letters = data['letter']
targets = data['label'].values
tensors = data_to_tensor(image_names)

In [17]:
X = tensors.astype("float32")/255
y = targets
y = keras.utils.to_categorical(y-1, 33)


In [18]:
X_train_whole, X_test, y_train_whole, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train_whole, y_train_whole, test_size=0.1, random_state=1)

In [19]:
print(X_train.shape)
datagen = ImageDataGenerator(rotation_range=10,  zoom_range = 0.1, width_shift_range=0.1,  height_shift_range=0.1)
print(datagen.fit(X_train))


(11493, 32, 32, 3)
None


In [20]:
deep_RU_model = Sequential()

deep_RU_model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (32,32,3)))
deep_RU_model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
deep_RU_model.add(MaxPooling2D(pool_size=(2,2)))
deep_RU_model.add(Dropout(0.25))


deep_RU_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
deep_RU_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
deep_RU_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
deep_RU_model.add(Dropout(0.25))


deep_RU_model.add(Flatten())
deep_RU_model.add(Dense(256, activation = "relu"))
deep_RU_model.add(Dropout(0.3))
deep_RU_model.add(Dense(33, activation = "softmax"))

In [22]:
deep_RU_model.compile(loss="categorical_crossentropy", optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),metrics=["accuracy"])
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0000001)
history = deep_RU_model.fit(datagen.flow(X_train,y_train, batch_size=90), validation_data = (X_val, y_val),epochs=139, callbacks=[learning_rate_reduction, es, mc])

Epoch 1/139
128/128 [==============================] - ETA: 0s - loss: 3.4862 - accuracy: 0.0378
Epoch 00001: val_accuracy improved from -inf to 0.05399, saving model to best_model.h5
128/128 [==============================] - 47s 370ms/step - loss: 3.4862 - accuracy: 0.0378 - val_loss: 3.4531 - val_accuracy: 0.0540
Epoch 2/139
128/128 [==============================] - ETA: 0s - loss: 3.4272 - accuracy: 0.0472
Epoch 00002: val_accuracy improved from 0.05399 to 0.06338, saving model to best_model.h5
128/128 [==============================] - 47s 366ms/step - loss: 3.4272 - accuracy: 0.0472 - val_loss: 3.3505 - val_accuracy: 0.0634
Epoch 3/139
128/128 [==============================] - ETA: 0s - loss: 3.3865 - accuracy: 0.0621
Epoch 00003: val_accuracy did not improve from 0.06338
128/128 [==============================] - 44s 341ms/step - loss: 3.3865 - accuracy: 0.0621 - val_loss: 3.7591 - val_accuracy: 0.0297
Epoch 4/139
128/128 [==============================] - ETA: 0s - loss: 3.34

128/128 [==============================] - ETA: 0s - loss: 0.4880 - accuracy: 0.8530
Epoch 00027: val_accuracy improved from 0.91784 to 0.93036, saving model to best_model.h5
128/128 [==============================] - 48s 378ms/step - loss: 0.4880 - accuracy: 0.8530 - val_loss: 0.2301 - val_accuracy: 0.9304
Epoch 28/139
128/128 [==============================] - ETA: 0s - loss: 0.4658 - accuracy: 0.8543
Epoch 00028: val_accuracy improved from 0.93036 to 0.93349, saving model to best_model.h5
128/128 [==============================] - 50s 389ms/step - loss: 0.4658 - accuracy: 0.8543 - val_loss: 0.2316 - val_accuracy: 0.9335
Epoch 29/139
128/128 [==============================] - ETA: 0s - loss: 0.4542 - accuracy: 0.8607
Epoch 00029: val_accuracy improved from 0.93349 to 0.93740, saving model to best_model.h5
128/128 [==============================] - 48s 378ms/step - loss: 0.4542 - accuracy: 0.8607 - val_loss: 0.2214 - val_accuracy: 0.9374
Epoch 30/139
128/128 [=========================

128/128 [==============================] - ETA: 0s - loss: 0.2701 - accuracy: 0.9158
Epoch 00053: val_accuracy did not improve from 0.95540
128/128 [==============================] - 55s 431ms/step - loss: 0.2701 - accuracy: 0.9158 - val_loss: 0.1557 - val_accuracy: 0.9546
Epoch 54/139
128/128 [==============================] - ETA: 0s - loss: 0.2711 - accuracy: 0.9147
Epoch 00054: val_accuracy did not improve from 0.95540
128/128 [==============================] - 53s 418ms/step - loss: 0.2711 - accuracy: 0.9147 - val_loss: 0.1551 - val_accuracy: 0.9538
Epoch 55/139
128/128 [==============================] - ETA: 0s - loss: 0.2770 - accuracy: 0.9140
Epoch 00055: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 00055: val_accuracy did not improve from 0.95540
128/128 [==============================] - 52s 406ms/step - loss: 0.2770 - accuracy: 0.9140 - val_loss: 0.1567 - val_accuracy: 0.9546
Epoch 56/139
128/128 [==============================] - ETA: 0s - loss:

KeyboardInterrupt: 

In [32]:
saved_model = load_model('best_model.h5')
_,test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

0.954193115234375
